In [126]:
import pandas as pd
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  

In [127]:
def processdf(df):
    df = df.fillna('~') 
    df.QTEXT = df.QTEXT.apply( lambda x: x.replace('~',',') )
    df.QTEXT = df.QTEXT.apply( lambda x: x.replace('" ','').replace('"','') )
    return df

In [128]:
df = pd.read_csv(fr'C:\Users\Tim\Documents\20-01\metrics.csv') 
df = processdf(df)  
df_mas = df

In [129]:
sql = ''
aspx = ''  
dlist = []
for i, r in df[:].iterrows():
    QPK = QPK + 1 
    r.ID = str(r.ID)
    m = str(r.ID).replace('.0','.1') 
    dlist.append(r.to_dict())
    dlist.append({'ID':f'{m}','QTEXT':'Notes'})
df_mas = pd.DataFrame(dlist);     
df_mas["SQL"] = ""
df_mas["ASPX"] = ""
df_mas["QPK"] = ""
df_mas[0:3]

,ID,QTEXT,SQL,ASPX,QPK
0,1.0,Were the trust boundaries and associated asse...,,,
1,1.1,Notes,,,
2,2.0,Was suspicious account activity associated wi...,,,


In [130]:
sql = ''
QPK = 23500
so = 0
for i, r in df_mas[:].iterrows():
    QPK=QPK+1 
    so=so+1
    df_mas.loc[i, 'QPK']= str(QPK)
    df_mas.loc[i, 'SQL'] = f"({QPK}, @FormName, @PK_QGroup, {so}, N'{r.ID}', NULL, NULL, NULL, 18, 1, NULL, 0, NULL, NULL, NULL, N'{r.QTEXT.strip()}',  NULL),"
    if '15.1' in r.ID or i== '46.1' in r.ID:
        so=0
        QPK=QPK+50
df_mas[0:3]

,ID,QTEXT,SQL,ASPX,QPK
0,1.0,Were the trust boundaries and associated asse...,"(23501, @FormName, @PK_QGroup, 1, N'1.0', NULL...",,23501
1,1.1,Notes,"(23502, @FormName, @PK_QGroup, 2, N'1.1', NULL...",,23502
2,2.0,Was suspicious account activity associated wi...,"(23503, @FormName, @PK_QGroup, 3, N'2.0', NULL...",,23503


In [131]:
aspx = '' 
s =''
sordr = 0
for i, r in df_mas[:].iterrows(): 
    if '.0' in r.ID:
        s = open(f'C:\\_som\\_src\\PICK.aspx', 'r').read()
    else:
        s = open(f'C:\\_som\\_src\\TXT.aspx', 'r').read() 
    metric = str(r.ID).replace('.','_')
    s = s.replace('{0}',str(r.QPK)).replace('{1}',str(metric)).replace('{4}',str(metric)+ ' : ' +str(r.QPK)+ '\n\t' +str(r.QTEXT)).replace('{5}','')
    s = s.replace('{2}',str(160))
    df_mas.loc[i, 'ASPX'] = s 
df_mas.to_csv(r"C:\Users\Tim\Documents\20-01\df_mas.csv", index='false') 
df_mas[0:3]

,ID,QTEXT,SQL,ASPX,QPK
0,1.0,Were the trust boundaries and associated asse...,"(23501, @FormName, @PK_QGroup, 1, N'1.0', NULL...",<!-- \n 1_0 : 23501\n\t Were the trust boun...,23501
1,1.1,Notes,"(23502, @FormName, @PK_QGroup, 2, N'1.1', NULL...",<!-- \n 1_1 : 23502\n\tNotes\n--->\n<tr id=...,23502
2,2.0,Was suspicious account activity associated wi...,"(23503, @FormName, @PK_QGroup, 3, N'2.0', NULL...",<!-- \n 2_0 : 23503\n\t Was suspicious acco...,23503


In [133]:
sql=''
aspx=''
for i, r in df_mas[:].iterrows(): 
    sql=sql+'\n'+r.SQL
    aspx=aspx+'\n\n'+r.ASPX
    
with open(r'C:\Users\Tim\Documents\20-01\script.sql', 'w') as file:
  file.write(sql)
file.close()
with open(r'C:\Users\Tim\Documents\20-01\aspx.aspx', 'w') as file:
  file.write(aspx)
file.close()

df_mas[0:5]

,ID,QTEXT,SQL,ASPX,QPK
0,1.0,Were the trust boundaries and associated asse...,"(23501, @FormName, @PK_QGroup, 1, N'1.0', NULL...",<!-- \n 1_0 : 23501\n\t Were the trust boun...,23501
1,1.1,Notes,"(23502, @FormName, @PK_QGroup, 2, N'1.1', NULL...",<!-- \n 1_1 : 23502\n\tNotes\n--->\n<tr id=...,23502
2,2.0,Was suspicious account activity associated wi...,"(23503, @FormName, @PK_QGroup, 3, N'2.0', NULL...",<!-- \n 2_0 : 23503\n\t Was suspicious acco...,23503
3,2.1,Notes,"(23504, @FormName, @PK_QGroup, 4, N'2.1', NULL...",<!-- \n 2_1 : 23504\n\tNotes\n--->\n<tr id=...,23504
4,3.0,Were all credentials stored on or used by Sol...,"(23505, @FormName, @PK_QGroup, 5, N'3.0', NULL...",<!-- \n 3_0 : 23505\n\t Were all credential...,23505


In [134]:
showrow = """\
    Protected _ShowRow_{0} As String
    Public Property ShowRow_{0}() As String
        Get
            Return _ShowRow_{0}
        End Get
        Set(ByVal value As String)
            _ShowRow_{0} = value
        End Set
    End Property
    
"""

In [135]:
sr=''
for i,r in df_mas.iterrows():
    sr = sr + showrow.replace('{0}', r.ID.replace(".","_"))
print(sr)

    Protected _ShowRow_1_0 As String
    Public Property ShowRow_1_0() As String
        Get
            Return _ShowRow_1_0
        End Get
        Set(ByVal value As String)
            _ShowRow_1_0 = value
        End Set
    End Property
    
    Protected _ShowRow_1_1 As String
    Public Property ShowRow_1_1() As String
        Get
            Return _ShowRow_1_1
        End Get
        Set(ByVal value As String)
            _ShowRow_1_1 = value
        End Set
    End Property
    
    Protected _ShowRow_2_0 As String
    Public Property ShowRow_2_0() As String
        Get
            Return _ShowRow_2_0
        End Get
        Set(ByVal value As String)
            _ShowRow_2_0 = value
        End Set
    End Property
    
    Protected _ShowRow_2_1 As String
    Public Property ShowRow_2_1() As String
        Get
            Return _ShowRow_2_1
        End Get
        Set(ByVal value As String)
            _ShowRow_2_1 = value
        End Set
    End Property
    
    Protecte

In [ ]:
import nltk
stop_words = set(stopwords.words('english'))  